<img src="xlogo.png" alt="Transposition of a vector" title="Vector transposition" width="900" style="float: left;">

Dans ce notebook, nous allons démontrer de manière rigoureuse la valorisation des instruments financiers tels que les caps et les floors ( dérivés de taux ) en utilisant les outils du calcul stochastique. Cette démonstration s'adresse à des lecteurs possédant une solide compréhension des taux d'intérêt. À cet égard, le premier chapitre du livre de Brigo et Mercurio, intitulé "Interest Rate Models: Theory and Practice", est une référence incontournable pour approfondir les concepts liés aux taux et aux instruments dérivés tels que les caps et les floors.

Nous adopterons les notations introduites par Brigo et Mercurio, que nous jugeons particulièrement pertinentes et générales. Sur cette base, nous apporterons des modifications reflétant nos propres compréhensions et recherches, afin d'offrir une perspective enrichie sur la valorisation des caps et des floors.

Dans un second temps, nous implémenterons ces méthodes en Python en utilisant la programmation orientée objet. 

#### Définition d'un cap et d'un floor

Un **cap** (plafond) est un instrument financier dérivé qui protège l'acheteur contre une hausse des taux d'intérêt au-delà d'un certain niveau prédéterminé, appelé taux d'exercice ou "strike". Il se compose d'une série d'options d'achat européennes sur taux d'intérêt, appelées "caplets", chacune correspondant à une période spécifique. À la fin de chaque période, si le taux d'intérêt de référence dépasse le taux d'exercice, l'acheteur du cap reçoit un paiement compensatoire. Les caps sont couramment utilisés par les emprunteurs à taux variable pour se protéger contre une augmentation excessive des taux d'intérêt.

À l'inverse, un **floor** (plancher) est un dérivé financier qui protège l'acheteur contre une baisse des taux d'intérêt en dessous d'un certain niveau prédéterminé. Il est constitué d'une série d'options de vente européennes sur taux d'intérêt, appelées "floorlets". À la fin de chaque période, si le taux d'intérêt de référence est inférieur au taux d'exercice, l'acheteur du floor reçoit un paiement compensatoire. Les floors sont souvent utilisés par les investisseurs ou les prêteurs pour se prémunir contre une diminution des revenus due à une baisse des taux d'intérêt.

Ces instruments permettent aux participants du marché de gérer le risque associé aux fluctuations des taux d'intérêt, en établissant des limites supérieures (caps) ou inférieures (floors) aux taux auxquels ils sont exposés. 

#### Cap Pricing : A step by step guide 


- Cap

It can be considered as a payer Interest Rate Swap where the payment are executed only when they are positives. Think of it as a we enter in a swap contract where we pay the fixed interest rate only if the payoff is positive! In other words, a cap is a contract in which the investor is compensated when the reference interest rate (namely the LIBOR or the floating rate) exceeds a fixed interest rate (interest rate strike or cap rate). Its payoff is:

$$
\sum_{i=\alpha+1}^\beta D\left(t, T_i\right) N \tau_i\left(L\left(T_{i-1}, T_i\right)-K\right)^{+}
$$

And its value under a risk neutral framework is:

$$
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i \mathbb{E}^{\mathbb{Q}}\left(D\left(t, T_i\right)\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right)
$$

How to price such a derivative product? Here are the main steps:
- A cap can be considered as sum of caplets. A caplet is a contract which pays the minimum between two interest rates: a floating rate between $T_{i-1}$ and $T_i$ namely $L\left(T_{i-1}, T_i\right)$ and a fixed interest rate $K$. Its payoff is

$$
D\left(t, T_i\right) N \tau_i\left(L\left(T_{i-1}, T_i\right)-K\right)^{+}
$$

- The price of a caplet under risk neutral framework is :

$$
C p(t)=N \tau_i \mathbb{E}^{\mathbb{Q}}\left(D\left(t, T_i\right)\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right)
$$

- Our main goal is to find the value of $C p(t)$. But, at this stage, we face one major problem. Indeed, interest rate are stochastic since we are dealing with interest rates derivatives ! When interest rates are stochastic, then the main problems faced during pricing are:
- The correlation between the stochastic discount factor $D\left(t, T_i\right)$ and the payoff $\left(L\left(T_{i-1}, T_i\right)-K\right)^{+}$
- Interest rates are not deterministic which means that the stochastic discount factor cannot be out of the conditional expectation!

To overcome this problem, we will use what we call the forward-neutral probability measure. It is the probablility under which, forward price of assets in our economy $\frac{S_t}{B(t, T)}$ (where $B(t, T)$ is the zero-coupon bond of maturity $T$ ) are martingales by considering $B(t, T)$ as the numeraire asset.

So, using Girsanov theorem and the change of numeraire asset formula we obtain the following formula:

$$
C p(t)=N \tau_i B\left(t, T_i\right) \mathbb{E}_{T_i}^{\mathbb{B}}\left(\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right)
$$

where $\mathbb{E}_{T_i}^{\mathbb{B}}$ represent the forward neutral probability measure, by the consdering the zero-coupon $B$ as numeraire and for time maturity $T_i$ since we want to price our caplet from time $T_i$ to $t$.
In other words, for a forward-neutral probability measure, the price of a derivative product is just the expectation of its payoff discounted by the zero-coupon bond and not the deterministic discount factor as we saw in the classic Black-Scholes model. The main advantages we have is the fact that under forward neutral probability measure, the zero-coupon bond is measurable and then, it can be out-of-the expectation.

- After this important stage, our goal is to compute the conditional expectation:

$$
\mathbb{E}_{T_i}^{\mathbb{B}}\left(\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right)
$$

A caplet can be seen as a call option whose the undelying asset is the Floating/Libor rate. We previously saw that in order to find the price of the option, we must have a probability distribution for the underlying asset. In the risk neutral framework, when the underlying is a stock, then it follows a lognormal distribution. Unfortunately, in our case, our underlying is the Floating/Libor rate. If we want to price such an option, we must find a tractable model for the floating rate. In fact, we want to price the option in terms of the famous Black-Scholes framework.

The most straightforward strategy is to suppose that the Floating/Libor rate follows also a lognormal distribution. But a major question remains: What is the stochastic differential equation verified by $L\left(T_{i-1}, T_i\right)$ ?

Brace-Gatarek-Musiela (BGM) answered this question. Indeed, under strong conditions, they find that the forward libor rate denoted in our case by $L\left(t, T_{i-1}, T_i\right)$ follows a lognormal distribution. The forward libor rate is just a simply compound forward interest rate. The $c \quad$ ard libor rate is the interest rate for which the value of a forward rate agreement at time $t$ is nul. Its value is equal to:

$$
L\left(t, T_{i-1}, T_i\right)=\frac{1}{\tau_i} \times\left(\frac{B\left(t, T_{i-1}\right)}{B\left(t, T_i\right)}-1\right)
$$


- After introducing the BGM framework, the key step is to use the property of the forward-neutral probability measure. Indeed, we know that the forward neutral probability measure is the probability measure under which the forward price of assets are martingales. Since we are dealing with interest rates derivatives, our asset can easily be a zero-coupon bond. We will say that it is the probability measure under which for example $\frac{B\left(t, T_{i-1}\right)}{B\left(t, T_i\right)}$ is a martingale. Notice that we are working the $T_i$-forward neutral measure meaning that all our assets, even if they are before of after $T_i$ must be divided by $T_i$ (Principle of the change of numeraire asset).
Since $\frac{B\left(t, T_{i-1}\right)}{B\left(t, T_i\right)}$ is a martingale, then our libor forward rate $L\left(t, T_{i-1}, T_i\right)$ defined previously is also a martingale !
- Then we use the Brownian Motion Martingale representation theorem! It is the most important step since we will easily find the SDE of the libor forward rate. According to this theorem, there exists a unique measurable stochastic process named in our case $\nu\left(t, T_{i-1}\right)$ such that:

$$
d L\left(t, T_{i-1}, T_i\right)=\nu\left(t, T_{i-1}\right) L\left(t, T_{i-1}, T_i\right) d W_t^{\mathrm{B}}
$$

where $v\left(t, T_{i-1}\right)$ can be assimilated as the volatility of the libor forward rate.
BGM choose the $v\left(t, T_{i-1}\right)$ process to be deterministic an give an explicit expression for it. According to them:

$$
\nu\left(t, T_{i-1}\right)=\left(\frac{1+\tau_i L\left(t, T_{i-1}, T_i\right)}{\tau_i L\left(t, T_{i-1}, T_i\right)}\right) \times\left(\sigma\left(t, T_{i-1}\right)-\sigma\left(t, T_i\right)\right)
$$

where $\sigma$ is the volatility of the zero coupon bond.

- We can now easily solve this SDE since its volatility is deterministic and with the standard stochastic calculus rules, we easily find that $L\left(T_{i-1}, T_i\right)$ conditional on $L\left(t, T_{i-1}, T_i\right)$ has a log-normal distribution under our forward-neutral probability measure and is equal to:

$$
L\left(T_{i-1}, T_i\right)=L\left(t, T_{i-1}, T_i\right) \times \mathrm{e}^{-\frac{1}{2} \int_t^{T_{i-1}} v^2\left(s, T_{i-1}\right) \mathrm{d} s+\int_t^{T_{i-1}} v\left(s, T_{i-1}\right) \mathrm{d} W^{\mathrm{B}}(s)}
$$

where $\int_t^T \nu\left(s, T_{i-1}\right) \mathrm{d} W^{\mathbb{B}}(s) \sim \mathcal{N}$ orm $\left(0, \bar{\nu}_{t, T_{i-1}} \sqrt{T_{i-1}-t}\right)$ since $\nu\left(t, T_{i-1}\right)$ is deterministic.

- Recall that our main goal is to price the following:

$$
C p(t)=N \tau_i B\left(t, T_i\right) \mathbb{E}_{T_i}^{\mathbb{B}}\left(\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right)
$$

We now have a tractable model of our Libor rate. Indeed, we just saw that is follows a log-normal distribution. And we give its explicit representation ! We can now easily compute our expectation and then qive the value of a standard caplet. Usina classical Black model and
where



We now have a tractable model of our Libor rate. Indeed, we just saw that is follows a log-normal distribution. And we give its explicit representation! We can now easily compute our expectation and then give the value of a standard caplet. Using classical Black model and using the fact the distribution of the floating rate is lognormal (after computations), we find that:

$$
C p(t)=N \tau_i B\left(t, T_i\right)\left[L\left(t, T_{i-1}, T_i\right) \mathcal{N}\left(d_{+}\left(t, T_{i-1}\right)\right)-K \mathcal{N}\left(d_{-}\left(t, T_{i-1}\right)\right)\right]
$$

$$
\begin{aligned}
& d_{ \pm}\left(t, T_{i-1}\right)= \frac{1}{\bar{v}_{t, T_{i-1}} \sqrt{T_{i-1}-t}}\left[\ln \frac{L\left(t, T_{i-1}, T_i\right)}{K} \pm \frac{1}{2} \bar{v}_{t, T_{i-1}}^2\left(T_{i-1}-t\right)\right] \\
& \bar{v}_{t, T_{i-1}}^2=\frac{1}{T_{i-1}-t} \int_t^{T_{i-1}} v^2\left(s, T_{i-1}\right) \mathrm{d} s
\end{aligned}
$$

- A cap is a sum of caplets. So, the value of a cap in the Black-Scholes framework is then:

$$
\begin{gathered}
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i \mathbb{E}^{\mathbb{Q}}\left(D\left(t, T_i\right)\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right) \\
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i B\left(t, T_i\right) \mathbb{E}_{T_i}^{\mathbb{B}}\left(\left(L\left(T_{i-1}, T_i\right)-K\right)^{+} \mid \mathscr{F}_t\right) \\
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i B\left(t, T_i\right)\left[L\left(t, T_{i-1}, T_i\right) \mathcal{N}\left(d_{+}\left(t, T_{i-1}\right)\right)-K \mathcal{N}\left(d_{-}\left(t, T_{i-1}\right)\right)\right]
\end{gathered}
$$

#### Floor Pricing : A step by step guide 

- Floor

It can be considered as a receiver Interest Rate Swap where the payment are executed only when they are positives. Think of it as a we enter in a swap contract where we receive the fixed interest rate only if the payoff is positive ! In other words, a floor is a contract in which the investor is compensated when the reference interest rate (namely the LIBOR or the floating rate) doesn't exceed a fixed interest rate (interest rate strike or cap rate). Its payoff is:

$$
\sum_{i=\alpha+1}^\beta D\left(t, T_i\right) N \tau_i\left(K-L\left(T_{i-1}, T_i\right)\right)^{+}
$$

And its value under a risk neutral framework is:

$$
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i \mathbb{E}^{\mathbb{Q}}\left(D\left(t, T_i\right)\left(K-L\left(T_{i-1}, T_i\right)\right)^{+} \mid \mathscr{F}_t\right)
$$

A floor is a sum of floorlets. A Floorlet can be think of as a put whose the underlying is the Floating/Libor rate. By using the same arguments we previously use for the Caplets and then for cap, we can easily deduce the price of the Floor. Then, it is given by:

$$
\begin{gathered}
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i B\left(t, T_i\right) \mathbb{E}_{T_i}^{\mathbb{B}}\left(\left(K-L\left(T_{i-1}, T_i\right)\right)^{+} \mid \mathscr{F}_t\right) \\
V(t)=\sum_{i=\alpha+1}^\beta N \tau_i B\left(t, T_i\right)\left[K \mathcal{N}\left(-d_{-}\left(t, T_{i-1}\right)\right)-L\left(t, T_{i-1}, T_i\right) \mathcal{N}\left(-d_{+}\left(t, T_{i-1}\right)\right)\right]
\end{gathered}
$$

where

$$
\begin{gathered}
d_{ \pm}\left(t, T_{i-1}\right)=\frac{1}{\bar{v}_{t, T_{i-1}} \sqrt{T_{i-1}-t}}\left[\ln \frac{L\left(t, T_{i-1}, T_i\right)}{K} \pm \frac{1}{2} \bar{v}_{t, T_{i-1}}^2\left(T_{i-1}-t\right)\right] \\
\bar{v}_{t, T_{i-1}}^2=\frac{1}{T_{i-1}-t} \int_t^{T_{i-1}} v^2\left(s, T_{i-1}\right) \mathrm{d} s
\end{gathered}
$$

#### Ressources on Interest Rate Derivatives Modeling

<img src="fb.png" alt="Transposition of a vector" title="Vector transposition" width="500" style="float: left;">
<img src="sb_.png" alt="Transposition of a vector" title="Vector transposition" width="300" style="float: left;">

<a href="https://www.amazon.fr/Interest-Rate-Models-Practice-Inflation/dp/3662517434">Interest Rate Models - Theory and Practice</a> : Le premier chapitre est absolument essentiel pour comprendre la théorie des taux d'intérêt, les différents types de taux et les dérivés de taux d'intérêt tels que les swaps de taux, les caps, les floors et les swaptions, ainsi que les notations utilisées dans les démonstrations.

<a href="https://www.librairiedalloz.fr/livre/9782247214839-finance-de-marche-5e-edition-roland-portait-patrice-poncet/">Finance de Marché</a> : Les chapitres 16 et 17 sont très importants pour comprendre les produits de taux d'intérèts, le pricing des produits de base des taux comme des caps, floors, swaptions et l'intuition + des démonstrations digestes de ces derniers .